## 🔖목차
- [1. Selenium 기반 데이터 크롤링](#1.-Selenium-기반-데이터-크롤링)
  - [크롤링 수행](#크롤링-수행)
- [2. 데이터셋 취합](#2.-데이터셋-취합)
  - [데이터 병합](#데이터-병합)
  - [누락 행 확인](#누락-행-확인)
- [3. 텍스트 전처리](#3.-텍스트-전처리)

# 1. Selenium 기반 데이터 크롤링
- BigkindsCrawler 클래스 & 함수를 만들어서 웹 크롤링 작업을 자동화함

In [1]:
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import os
import time
import pandas as pd
import numpy as np
import re
import json


class BigkindsCrawler:
    def __init__(self, path, year):
        self.df = pd.read_csv(path, encoding="utf-8-sig")
        self.year = year
        # 각 월 별 날짜 수 (Hard-coded) & 각 월
        self.months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
        self.days = ["31", "28", "31", "30", "31", "30", "31", "31", "30", "31", "30", "31"]
        self.options = webdriver.ChromeOptions()
        self.crawled_df = pd.DataFrame(columns=["일자", "언론사", "제목", "URL", "본문"])

    # set the WebDriver options
    def set_driver_options(self):
        self.options.add_argument('--window-size=1920,1080')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        # set User-Agent for preventing access blocked
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)" +
                                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
        # prevent webdriver from closing immediately
        self.options.add_experimental_option("detach", True)
        # 크롬 브라우저가 직접적으로 열리지 않도록 설정
        self.options.add_argument('--headless')
        # 불필요한 이미지 로딩 없앰 (시간 단축)
        self.options.add_argument('--disable-logging')
        self.options.add_argument('--disable-images')

    # csv 파일 필요: publisher, keyword를 csv로 먹임
    # category: 통합 분류 (li: 정치=1, 경제=2, 사회=3, 국제=5)
    def executor(self, publisher, m, category, keyword):
        res = []

        start_day = self.year + "-" + self.months[m] + "-" + "01"
        end_day = self.year + "-" + self.months[m] + "-" + self.days[m]

        # webdriver 생성
        driver = webdriver.Chrome(options=self.options)
        driver.get("https://www.bigkinds.or.kr/v2/news/index.do")

        # 언론사 클릭
        pub = self.transform_publisher(publisher)
        driver.find_element(By.XPATH, f"//*[@id='category_provider_list']/li[{pub}]/span/label").click()
        time.sleep(0.5)

        # 기간 클릭 (배너)
        driver.find_element(By.XPATH, "//*[@id='collapse-step-1-body']/div[3]/div/div[1]/div[1]/a").click()
        # 기간 클릭 (1개월)
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[1]/span[3]/label").click()

        # 시작 날짜 클릭
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[2]/div/div[1]/img").click()
        start = driver.find_element(By.XPATH, "//*[@id='search-begin-date']")
        start.send_keys(Keys.CONTROL, 'a')
        start.send_keys(start_day)

        # 종료 날짜 클릭
        driver.find_element(By.XPATH, "//*[@id='srch-tab1']/div/div[2]/div/div[3]/img").click()
        end = driver.find_element(By.XPATH, "//*[@id='search-end-date']")
        end.send_keys(Keys.CONTROL, 'a')
        end.send_keys(end_day)
        time.sleep(0.5)

        # 통합 분류 클릭 (배너)
        # 그냥 클릭하면 페이지 로딩 시간 때문에 오류가 날 수 있어서 webdriver 기다림
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='collapse-step-1-body']/div[3]/div/div[2]/div[1]/a"))
        )
        element.click()

        # 통합 분류 (li: 정치=1, 경제=2, 사회=3, 국제=5)
        driver.find_element(By.XPATH, f"//*[@id='srch-tab3']/ul/li[{category}]/div/span[4]").click()
        time.sleep(1)

        # 키워드 입력: 오류 나지 않게 한 글자씩 입력함
        keyword_input = driver.find_element(By.XPATH, "//*[@id='total-search-key']")
        for k in keyword:
            keyword_input.send_keys(k)
            time.sleep(0.2)
        keyword_input.send_keys(Keys.RETURN)
        time.sleep(1)

        # 정확도순
        driver.find_element(By.XPATH, "//*[@id='select1']/option[2]").click()
        time.sleep(1)

        try:
            # 맨 위의 기사 클릭
            driver.find_element(By.XPATH, "//*[@id='news-results']/div[1]/div/div[2]").click()
            time.sleep(1)

            # "일자", "언론사", "제목", "URL", "본문"
            try:
                date = driver.find_element(By.XPATH,
                                           "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/div[1]/ul/li[1]").text
            except NoSuchElementException:
                date = "N/A"
            
            title = driver.find_element(By.XPATH, "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/h1").text

            # URL 오류 처리
            href_button = driver.find_element(By.XPATH,
                                              "//*[@id='news-detail-modal']/div/div/div[1]/div/div[1]/div[2]/div[1]/button[1]")

            if href_button.text == "기사원문":
                href = href_button.get_attribute("onclick")

                # ?가 포함되었을 경우, 쿼리 문자열이므로 뒤의 문자열은 삭제
                try:
                    url = re.search(r'https?://[^?]+', href).group()
                except AttributeError:
                    # URL이 매치되지 않는 경우, 예외 처리를 통해 http 이후의 문자열만 저장
                    url = re.search(r'https?://+', href).group()
            else:
                url = "N/A"

            paper = driver.find_element(By.XPATH, "//*[@id='news-detail-modal']/div/div/div[1]/div/div[2]")
            main_text = paper.text
        except NoSuchElementException:
            date = "N/A"
            publisher = "N/A"
            title = "N/A"
            url = "N/A"
            main_text = "N/A"

        res.append(date)
        res.append(publisher)
        res.append(title)
        res.append(url)
        res.append(main_text)

        driver.quit()

        print(res)
        return res

    # 월 단위별로 크롤링
    def crawling(self, MONTH):
        s_index = (MONTH - 1) * 16
        size = 16
        publishers = self.df.loc[s_index:s_index + size, "언론사"]

        categories = self.df.loc[s_index:s_index + size, "카테고리"]

        total_time = 0

        for i in range(s_index, s_index + size):
            rank_str = self.df.loc[i, "top-10 키워드"]
            rank_str = rank_str.replace("'", '"')

            # JSON 문자열을 파이썬 리스트로 변환
            data_list = json.loads(rank_str)
            data_list = data_list[:5]

            keywords = [item['name'] for item in data_list]

            publisher = publishers[i]
            category = categories[i] // 1000000

            # 변수 체크용
            print(f"CSV 행 = {i}")
            print(f"언론사 = {publisher}")
            print(f"카테고리 = {category}")

            for j, keyword in enumerate(keywords):
                s = time.time()
                print(
                    f"{'>>>>>  * Process: ' + str(MONTH) + 'th month ' + str(i * 5 + j + 1) + 'th/' + '960th *  <<<<<':^50}")
                self.crawled_df.loc[i * 5 + j, :] = self.executor(publisher, MONTH - 1, category, keyword)
                e = time.time()
                total_time += round(e - s, 2)
                print(f"누적 소요 시간: {total_time:.2f}")
                time.sleep(1)

    def transform_publisher(self, p):
        pub = 0
        if p == "경향신문":
            pub = 1
        elif p == "동아일보":
            pub = 4
        elif p == "조선일보":
            pub = 8
        elif p == "중앙일보":
            pub = 9
        elif p == "한겨레":
            pub = 10

        return pub

    def get_df(self):
        return self.crawled_df


## 크롤링 수행
- 실제 크롤링 수행하는 코드
- 연도를 입력하면 총 12달에 대한 기사 본문 텍스트를 각각 csv 파일로 저장해줌

In [2]:
YEAR = "2014" #👈여기에 크롤링할 연도를 입력해주세요.

crawler_2013 = BigkindsCrawler(f"topkeywords_{YEAR}.csv", YEAR)
crawler_2013.set_driver_options()  # 옵션 세팅

# 1 ~ 12월까지 크롤링하고, 각 월 별로 데이터 프레임을 만듭니다.
for Month in range(1, 13):
    crawler_2013.crawling(Month)
    dataframe = crawler_2013.get_df()
    # 월별로 csv 추출도 진행합니다.
    dataframe.to_csv(f"{YEAR}_{Month}.csv", encoding="utf-8-sig")
    print(f"{YEAR}_{Month}.csv complete! \n")
    time.sleep(1)


CSV 행 = 0
언론사 = 중앙일보
카테고리 = 1
  >>>>>  * Process: 1th month 1th/960th *  <<<<<  
['2014-01-11', '중앙일보', '전병헌 "북 인권 거론할 때 됐다"', "https://www.joongang.co.kr/article/13613393'", '여야, 북 인권법 논의 진전 가능성김한길 신년 회견 때도 언급 검토\n전병헌\n민주당 원내대표는 10일 “이제 남북 관계만 아니라 북한 인권도 거론할 시점”이라고 밝혔다. 전 원내대표는 이날 출입기자들과 가진 만찬 간담회에서 이같이 밝혔다. 김한길 민주당 대표도 오는 13일 신년 기자회견에서 북한인권법 문제를 거론하는 방안을 검토 중이다. 이에 따라 2월 임시국회에서는 새누리당이 요구해온 북한인권법에 대한 논의가 여야 간에 진행될 가능성이 높아졌다. 전 원내대표가 북한 인권을 언급한 데 대해 간담회에 배석했던 홍익표 의원은 “민주당이 새누리당에서 주장하는 북한인권법 문제를 그대로 수용할지를 논의하는 차원이 아니라 사형제 폐지에서처럼 남북의 공통 주제 등으로 북한 인권 문제를 시작할 수 있다는 의미”라고 설명했다.\u3000김 대표의 13일 기자회견 내용과 관련, 민주당의 핵심 관계자는 “북한 인권 문제에 대해선 민주당이 보다 전향적으로 접근할 필요성이 있다는 데 내부 의견이 모아지고 있다”며 “기자회견에 이를 담을지와 어떻게 담을지에 대해 논의 중”이라고 밝혔다. 이 관계자는 “북한인권 관련법과 함께 나라와 민족을 위해 희생했던 전몰 장병 등을 기리는 가칭 애국자법에 대해서도 기자회견을 통해 제안하는 방안을 검토하고 있다”고 덧붙였다.\u3000그동안 북한인권법 이슈는 새누리당이 공세적이었고, 민주당에선 언급을 꺼려 왔다. 민주당은 북한 인권을 거론하는 자체가 남북 관계에 불필요한 긴장을 야기하고, 북한 정권 타도를 주장하는 대북단체에 대한 지원론으로 바뀔 수 있다는 이유를 들어왔다. 그러나 비주류인 민주당 지도부가 북한 인권법 문제에 대해 전향적인 

누적 소요 시간: 148.85
 >>>>>  * Process: 1th month 10th/960th *  <<<<<  
['2014-01-22', '중앙일보', '[Think money] 미국 한파로 엔화 강세, 한국 증시엔 호재 기대', "https://www.joongang.co.kr/article/13701262'", '곽현수(/br)신한금융투자 연구원\n지구촌 곳곳에서 기상 이변이 나타나고 있다. 자신을 더럽힌 인간에게 지구가 본격적으로 범칙금을 물리기 시작했음을 보여주는 신호인지도 모르겠다. 해마다 되풀이되는 기상 이변은 이제 이변이 아닌 일상이 됐다. 미국도 일상화된 이변으로 큰 피해가 예상된다. 미국에 혹한이 몰아쳤다. 1주일 동안 계속된 한파는 103년 만에 나이애가라 폭포가 완전히 얼어붙는 진풍경을 만들어냈다. \u3000미국 경기에 이번 한파는 많은 영향을 끼쳤다. 우선 1월 미국 경기 및 고용 지표가 일시적으로 부진할 가능성이 커졌다. 현지 언론에 따르면 이번 한파는 미국 1분기 국내총생산(GDP)을 0.2%포인트 하락하게 하는 요인이 될 수 있다고 한다. 무엇보다 주목해야 할 바는 고용이다. 혹한의 추위는 인간의 노동 의지를 꺾어 놓는다. 추우면 밖에 나가기 싫다. 그것도 잘못하다 죽을 수 있는 추위라면 더욱 그렇다. 현재로만 놓고 보면 미국 1월 고용은 부진할 수밖에 없어 보인다.\u3000최근 미국 구글 트렌드에서도 실업 수당 신청 서류 검색 건수가 급증했다. 이 지표는 실제 미국 실업 수당 청구 건수와 밀접하게 움직이는 것으로 알려져 있다. 실제로 미국 내 고용 등 전반적인 경기 지표가 부진하다면 미 국채 금리의 상승세는 둔화될 가능성이 높다.\u3000혹한이 미국 경기 둔화로 이어질 경우 분명 악재다. 다만 최근 한국 증시와 관련해서는 다소 긍정적인 부분도 존재한다. 바로 엔-달러 환율의 상승세가 약화될 가능성을 높이는 변수이기 때문이다. 미국 금리 하락은 미국 달러의 약세 요인이다. 반면 반대쪽의 엔화에는 강세 요인이 된다.\u

누적 소요 시간: 259.80
 >>>>>  * Process: 1th month 17th/960th *  <<<<<  
['2014-01-11', '중앙일보', '[브리핑] 한빛부대, 일 자위대 탄약 18일 만에 반환', "https://www.joongang.co.kr/article/13613379'", '남수단에 파병된 한빛부대가 10일 남수단임무단(UNMISS)을 통해 빌린 일본 자위대의 탄약 1만 발을 반환했다. 한빛부대는 남수단 내전이 재발하면서 지난해 12월 21일 UNMISS를 통해 일본 육상자위대의 5.56㎜ 소총 실탄 1만 발 등 탄약과 무기 등을 지원받았다. 그러나 한국군 창군 이래 처음으로 일본으로부터 탄약 지원을 받은 것을 두고 논란이 불거지자 18일 만에 이를 반환했다.']
누적 소요 시간: 275.21
 >>>>>  * Process: 1th month 18th/960th *  <<<<<  
['2014-01-30', '중앙일보', "[브리핑] 미국 조지아주 '동해' 명기 결의안 통과", "https://www.joongang.co.kr/article/13772257'", '미국 조지아주 의회가 한반도 동쪽 바다를 ‘동해(East Sea)’로 명기한 결의안을 28일(현지시간) 처리했다고 AP통신이 보도했다. 조지아주 상원은 이날 오전 전체회의를 열고 한국과 조지아주의 경제협력에 대한 평가를 담은 상원 결의안 798호를 만장일치로 통과시켰다.']
누적 소요 시간: 290.76
 >>>>>  * Process: 1th month 19th/960th *  <<<<<  
['2014-01-27', '중앙일보', '[사진] 이집트 시민혁명 3주년 시위', "https://www.joongang.co.kr/article/13741741'", "이집트 카이로 타히르 광장에서 25일(현지시간) 시민혁명 3주년을 맞아 반정부 시위가 열렸다. 이날 군부 찬반세력의 유혈충돌로 최소 29명이 사망하고 170여 명이 부상을 입었다. 광장에 모인 수많은 사람

누적 소요 시간: 436.67
 >>>>>  * Process: 1th month 28th/960th *  <<<<<  
['2014-01-07', '경향신문', 'SK, 고부가 ‘파라자일렌 공장’ 어떡하나', 'N/A', 'SK그룹이 석유화학제품인 파라자일렌(PX) 때문에 웃지도, 울지도 못하는 처지에 놓였다. 외국인투자촉진법(외촉법) 개정안이 국회를 통과해 울산 공장은 한숨을 돌렸지만, 인천 공장은 지방자치단체가 제동을 걸고나서 완공 시기가 불투명해졌다.\nSK이노베이션은 자회사인 SK인천석유화학과 SK종합화학을 통해 2012년부터 인천과 울산에 파라자일렌 공장을 증설하고 있다. 파라자일렌은 원유 정제과정에서 나오는 나프타를 분해한 물질로 페트병과 합성섬유의 원료로 쓰인다.\n파라자일렌은 최근 중국을 중심으로 합성섬유 수요가 급증하면서 매년 7%가량의 성장이 예상될 정도로 ‘황금알을 낳은 거위’로 인식돼왔다. 정유업계는 정제 마진이 줄면서 수익이 악화되자 3~4년 전부터 경쟁적으로 파라자일렌 증산에 뛰어들었다.\n\n인천 서구청은 6일 SK인천석유화학에 공사 일시 중단을 명령할 계획이라고 밝혔다. SK인천석유화학은 인천 서구 원창동에 11만5700여㎡ 규모의 공장을 짓고 있다. 국민연금 등의 투자를 받아 1조6200억원을 들여 현재 공정의 90%가량이 진행돼 완공을 앞두고 있다.\n하지만 지난해 초 주민들이 유동성 물질 배출 등 환경오염을 문제삼으면서 공장 증설이 차질을 빚고 있다. 지난달에는 인천시가 서구청을 감사해 해당 공장이 공작물을 무단 축조하고 제조시설 면적신고를 누락했음에도 공사를 승인했다고 발표했다. 인천시는 공사 중단을 권고했고, 서구청이 이를 받아들여 이날 공사 중단을 명령하기로 한 것이다. 서구청은 그러나 공사 중단 기간은 구체적으로 밝히지 않았다.\nSK그룹은 “공사 중단에 따른 손실액만 매달 1100억원쯤 되고 공사 중단 기간이 길어질 경우 SK인천석유화학까지 문을 닫게 될 수도 있다”며 “손해배상 소송이나 행정소송 등 다양한 대응책을

누적 소요 시간: 580.28
 >>>>>  * Process: 1th month 37th/960th *  <<<<<  
['2014-01-26', '경향신문', '케네디 주일미대사 ‘돌고래 포획 반대’ 트윗…일본에서 미묘한 ‘반감’', 'http://news.khan.co.kr/kh_news/khan_art_view.html', '일본의 돌고래 포획을 비판한 캐롤라인 케네디 주일 미국대사에 대해 일본에서 반감이 일어나고 있다고 인터내셔널뉴욕타임스(INYT)가 24일(현지시간) 보도했다.\n\n이 신문은 케네디 대사가 일본 국민들의 열렬한 환영을 받으면서 일본땅을 밟았지만, ‘돌고래 포획 반대’ 입장 표명 이후 변화하고 있는 분위기를 소개했다.\n\n케네디 대사는 지난 19일 태평양 연안 와카야마현 다이지초에서 이뤄지는 돌고래 포획에 대해 공식 트위터를 통해 우려를 표시했다. 그는 트위터에 “비인간적 돌고래 도살에 깊은 우려를 나타내고 미 정부는 이에 반대한다”고 밝혔다. 이후 스가 요시히데(菅義偉) 일본 관방장관이 기자회견에서 “돌고래 포획은 일본의 전통적인 낚시법의 하나이며 법령에 근거해 적절히 실시되고 있다”며 반박했다.\n\n아베 신조 일본 총리도 최근 CNN과의 인터뷰에서 “돌고래 사냥은 마을의 문화에서 비롯된 전통적 방식의 사냥법으로, 주민들이 생계를 꾸려가는 수단”이라고 밝혔다.\n\nINYT는 이런 일본 정부의 입장에 공감하는 일본 국민들은 케네디 대사에게 곱지 않은 시선을 보내고 있다고 전했다.\n\n<디지털뉴스팀>']
누적 소요 시간: 595.96
 >>>>>  * Process: 1th month 38th/960th *  <<<<<  
['2014-01-16', '경향신문', '이스라엘 국방 “케리 미 국무장관 노벨상 받고 빨리 떠나야” 원색 비판 파문', 'N/A', '이스라엘 강경파 모셰 야알론 국방장관이 최대 우방이자 자국과 팔레스타인의 평화협정을 중재하는 미국에 원색적인 비난을 쏟아냈다.\n야알론은 평화회담을 주도하는 존 케리 미국 

누적 소요 시간: 741.61
 >>>>>  * Process: 1th month 47th/960th *  <<<<<  
['2014-01-09', '동아일보', '[알립니다]‘2014 국가 소비자중심 브랜드 대상’에 도전하세요', "https://www.donga.com/news/article/all/20140108/60054027/1'", '[사진기사]']
누적 소요 시간: 757.85
 >>>>>  * Process: 1th month 48th/960th *  <<<<<  
['2014-01-27', '동아일보', '[기업이 미래다]에쓰오일, 대규모 시설투자… 종합 에너지사 도약', "https://www.donga.com/news/article/all/20140125/60395750/1'", '에쓰오일(S-OIL)은 올해 핵심 경영전략으로 ‘차세대 성장 동력 확보를 위한 시설투자’를 꼽았다. 에쓰오일이 현재 석유사업 부문에서 뛰어난 경쟁력을 자랑할 수 있는 것도 장기적 안목을 바탕으로 한 시설투자를 지속한 덕분이라는 것이 내부 판단이다.\n\n이는 나세르 알마하셰르 대표의 신년사에도 잘 나타나 있다. 그는 “에쓰오일은 올해 대규모 시설투자 프로젝트를 검토 중이며 이는 정유·윤활·석유화학 사업을 아우르는 수익성 높은 종합 에너지 회사를 만들기 위한 첫걸음이 될 것이다”라고 밝혔다.\n\n올해 에쓰오일은 연구개발(R&D) 분야를 한층 더 강화하기 위해서 첨단연구소(Technical Service & Development Center) 건립을 추진한다. 서울 마곡산업단지에 약 3만 m² 용지를 확보할 예정인데 16일 서울시 마곡산단 정책심의위원회 최종심의를 통과해 입주협상 대상자로 선정됐다. 에쓰오일은 첨단 연구소가 자동차부터 가전제품, 그리고 정보기술(IT)과 생명공학(BT) 등 파생 제품 종류가 많아 성장 잠재력이 높은 석유화학 하류부문에서 크게 기여해줄 것으로 기대하고 있다.\n\n에쓰오일이 과감한 시설 투자에 자신감을 보이는 것은 2011년 1조300

누적 소요 시간: 900.99
 >>>>>  * Process: 1th month 57th/960th *  <<<<<  
['2014-01-25', '동아일보', '日 “동해병기 말라” 美주지사 ‘협박’', "https://www.donga.com/news/article/all/20140125/60392435/1'", '주미대사 “경제관계 악영향” 편지… 버지니아주 상원, 법안 통과시켜\n사사에 겐이치로 주미 일본대사가 동해와 일본해 병기 의무화 법안을 저지하기 위해 지난해 말 테리 매콜리프 미국 버지니아 주지사 당선자에게 편지를 보내 “법안이 통과되면 일본과 버지니아 주의 경제관계에 악영향이 올 수 있다”고 사실상 협박을 했다고 미국 워싱턴포스트(WP)가 23일 폭로했다. 이는 국제 관행에 어긋나는 외교적인 결례여서 파장이 커질 것으로 보인다.\n\nWP가 입수한 편지에 따르면 사사에 대사는 “나는 일본과 버지니아 사이의 긍정적인 협력관계와 강력한 경제적 유대가 훼손될 것이 우려스럽다”며 △5년 내 일본의 버지니아 직접투자 10억 달러(약 1조600억 원) △250개 현지 일본 회사 △버지니아 생산품 수백만 달러어치의 대(對)일본 시장 수출 등을 적시해 매콜리프 주지사를 압박했다.\n\n사사에 대사는 또 22일 주지사 사무실을 찾아가 직접 법안 저지를 요청한 것으로 알려졌다. 일본 대사관은 로비스트 4명을 고용해 조직적으로 법안 통과 방해 공작을 벌였다.\n\n그러나 버지니아 주 상원은 23일 전체회의에서 찬성 31표, 반대 4, 기권 3표로 동해 병기 법안을 통과시켰다. 하원에 계류된 유사 법안이 통과되면 조율과정을 거쳐 주지사가 최종 서명한 뒤 7월 1일부터 공식 발효된다. 그러나 일본이 총력 로비를 펼칠 것으로 보여 난항이 예상된다. 한편 일본 정부 대변인인 스가 요시히데(菅義偉) 관방장관은 24일 “일본해라는 명칭이 국제적으로 확립된 유일한 명칭”이라며 “적절히 대응해 나갈 생각”이라고 밝혔다.\n\n워싱턴=신석호 특파원 kyle@donga.com']

누적 소요 시간: 1028.91
 >>>>>  * Process: 1th month 65th/960th *  <<<<<  
['2014-01-05', '한겨레', '윤여준, ‘결별했던 안철수와 왜 재결합?’ 질문에…', "http://www.hani.co.kr/arti/politics/assembly/618348.html'", '안철수 무소속 위원이 창당 준비 실무 기구인 새정치추진위원회(새정추) 공동위원장에 윤여준 전 환경부 장관을 영입했다.\n\n안 의원은 11일 서울 여의도 새정추 사무실에서 기자회견을 갖고 “지금은 경륜과 지혜가 무엇보다 중요한데, 윤여준 위원장님은 한국 정치의 현장을 두루 경험하셨고 그 경험은 무엇보다 소중하다”며 윤 위원장의 선임을 알렸다.\n\n윤 위원장은 “‘새 정치’는 우리 모두의 소망이자 국민의 열망으로 그게 시대적 요청이고 누구도 거역할 수 없는 역사의 명령이다”며 위원장을 맡게 된 이유를 설명했다. 그는 “현재 1세대 정치(권위주의 리더십)와 2세대 정치(민주화 시기)가 부딪히고 있는데 안철수 의원의 등장은 역사적 필연”이라고 덧붙였다.\n\n윤 위원장은 과거 안 의원과 결별했다 다시 함께 한 것에 대해 “저는 결별이라고 생각 않는다. 서울시장 출마를 생각하셨다가 안 하시니까 그 일을 중단하게 된 것이다”며 “지난 8월 초 이후 안 의원이 끈질기게 요청했고, 새 정치가 큰 소망이었고 안 의원의 새 정치가 역사적 명령이라면 아무리 능력이 부족해도 작은 능력이나마 보태는 게 맞다고 생각해 이 자리에 섰다”고 밝혔다.\n\n윤 위원장은 ‘과거 안 의원과 현재의 안 의원이 어떻게 달라졌느냐’는 질문에 “대단히 집요해졌고, 굉장히 인상이 강인해졌다”고 답했다.\n\n이승준기자 gamja@hani.co.kr']
누적 소요 시간: 1044.79
CSV 행 = 13
언론사 = 한겨레
카테고리 = 2
 >>>>>  * Process: 1th month 66th/960th *  <<<<<  
['2014-01-07', '한겨레', '대·중소

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: headless chrome=118.0.5993.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C1168EF2+54786]
	(No symbol) [0x00007FF6C10D5612]
	(No symbol) [0x00007FF6C0F8A64B]
	(No symbol) [0x00007FF6C0F8F116]
	(No symbol) [0x00007FF6C0F9115F]
	(No symbol) [0x00007FF6C0F91250]
	(No symbol) [0x00007FF6C0FCFC79]
	(No symbol) [0x00007FF6C0FCDC08]
	(No symbol) [0x00007FF6C0FCCCC3]
	(No symbol) [0x00007FF6C0FC29CF]
	(No symbol) [0x00007FF6C0FEBE6A]
	(No symbol) [0x00007FF6C0FC22E6]
	(No symbol) [0x00007FF6C0FEC080]
	(No symbol) [0x00007FF6C1004D02]
	(No symbol) [0x00007FF6C0FEBC43]
	(No symbol) [0x00007FF6C0FC0941]
	(No symbol) [0x00007FF6C0FC1B84]
	GetHandleVerifier [0x00007FF6C14B7F52+3524194]
	GetHandleVerifier [0x00007FF6C150D800+3874576]
	GetHandl

In [3]:
crawler_2013.crawled_df

,일자,언론사,제목,URL,본문
0,2014-01-11,중앙일보,"전병헌 ""북 인권 거론할 때 됐다""",https://www.joongang.co.kr/article/13613393',"여야, 북 인권법 논의 진전 가능성김한길 신년 회견 때도 언급 검토\n전병헌\n민주..."
1,2014-01-20,중앙일보,[서소문 포럼] 새누리당의 허술한 위헌론과 비겁,https://www.joongang.co.kr/article/13680886',전영기(/br)논설위원\n새누리는 새로운 세상이란 뜻의 우리말입니다. 새누리당이 보...
2,2014-01-25,중앙일보,[브리핑] 의장 직속 헌법개정자문위 출범,https://www.joongang.co.kr/article/13733160',강창희 국회의장 직속의 헌법개정자문위원회가 24일 출범했다. 김철수 서울대 명예교수...
3,2014-01-15,중앙일보,[사진] 연세동문 새해 인사의 밤,https://www.joongang.co.kr/article/13641216',연세대 총동문회(회장 박삼구)는 14일 서울 소공동 롯데호텔 크리스탈볼룸에서 ‘20...
4,2014-01-25,중앙일보,[브리핑] 의장 직속 헌법개정자문위 출범,https://www.joongang.co.kr/article/13733160',강창희 국회의장 직속의 헌법개정자문위원회가 24일 출범했다. 김철수 서울대 명예교수...
...,...,...,...,...,...
62,2014-01-20,한겨레,2014년 1월 남북관계 일지,N/A,"2014년 1월 남북관계 일지1월1일김정은 조선노동당 제1비서, 북-남 관계 개선 ..."
63,2014-01-29,한겨레,울산시장 /새누리 절대 우위 속 야권연대 관심,http://www.hani.co.kr/arti/society/area/621950...,새누리당이 절대우위인 영남권에서 울산은 2010년 6월 지방선거와 2011년 4월 ...
64,2014-01-05,한겨레,"윤여준, ‘결별했던 안철수와 왜 재결합?’ 질문에…",http://www.hani.co.kr/arti/politics/assembly/6...,안철수 무소속 위원이 창당 준비 실무 기구인 새정치추진위원회(새정추) 공동위원장에 ...
65,2014-01-07,한겨레,대·중소기업 대졸신입 연봉 1127만원 차이,http://www.hani.co.kr/arti/economy/working/618...,대기업과 중소기업 간 대졸 신입사원의 평균연봉이 1000만원 이상 차이가 나는 것으...


# 2. 데이터셋 취합

## 데이터 병합
- 월별로 수집한 데이터셋을 전부 하나로 병합

In [ ]:
import pandas as pd
import numpy as np

# 월별 데이터프레임을 저장할 공간
dataframes = []

# 2013년부터 2022년까지 불러오기
for year in range(2013, 2023):
    for month in range(1, 13):
        filename = f"{year}_{month}.csv"
        df = pd.read_csv(filename, encoding="utf-8-sig")
        dataframes.append(df)
        

# 행을 기준으로 결합
combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
len(dataframes) # 12개월 * 9년치 = 84 맞음

In [ ]:
combined_df.head(10)

In [ ]:
combined_df = combined_df[['언론사','제목','본문','일자','URL']] # 필요한 컬럼만 남김
combined_df

➡️1년에 960행 * 10년치 = 9,600 행 맞음!

In [ ]:
# '카테고리' 컬럼 다시 추가 (by 재언)
combined_df = pd.read_csv("9600 카테고리.csv", encoding="utf-8-sig")
combined_df

In [ ]:
count_politics = combined_df['카테고리'].value_counts().get('국제', 0)
count_politics

In [ ]:
combined_df = combined_df[['언론사','제목','카테고리','본문','일자','URL']]
combined_df

## 누락 행 확인

In [ ]:
# '본문' column에 누락된 경우 있는지 확인
missing_rows = combined_df[combined_df.iloc[:, 3].isna()]

missing_rows = pd.DataFrame(missing_rows)
missing_rows

In [ ]:
missing_rows.shape

In [ ]:
combined_df = combined_df.dropna(subset=['본문'])
combined_df

# 3. 텍스트 전처리
- 한국어 텍스트 난이도를 평가하는 데에 방해가 되는 요소(특수기호, 한자 및 일본어, 이메일 및 사이트 주소, 각종 괄호)를 제거

In [ ]:
import pandas as pd
import os
os.chdir('C:/Users/simon/PythonWorkspace/Psat_Datamining')
import warnings
warnings.filterwarnings('ignore')

import regex as re

In [ ]:
def cleaning_text(text):
    # text가 문자열이 아니면 그대로 반환
    if not isinstance(text, str):
        return text
    # (/br)로 나오는 경우 제거
    text = text.replace('(/br)', ' ')
    # 특수 문자 지정 후 제거
    text = re.sub(r'[☞▶◆#⊙※△▽▼□■◇◎☎○]+', ' ', text, flags=re.UNICODE)
    text = re.sub(r'〃', ' ', text)  
    # 한자 및 일본어 제거
    text = re.sub(r'[\p{Script=Hiragana}\p{Script=Katakana}\p{Script=Han}]+', ' ', text, flags=re.UNICODE)
    # 이메일 주소 제거
    text = re.sub(r'\S+@\S+', ' ', text)
    # 사이트 주소 제거(www. 으로 시작하고 .kr로 끝나는 경우)
    text = re.sub(r'www\..+\.kr', ' ', text)
    
    # 대괄호로 둘러싸인 내용을 삭제 (10글자 미만인 경우는 삭제, 10글자 이상인 경우는 유지)
    text = re.sub(r'\[([^\]]{1,9})\]', ' ', text)
    text = re.sub(r'\[([^\]]{10,})\]', r'\1', text)
    # "<...>"로 둘러싸인 내용을 삭제
    text = re.sub(r'<[^>]*>', ' ', text)
    # 소괄호 안에 아무런 내용도 없으면 삭제
    text = re.sub(r'\(\s*\)', ' ', text)
    
    # 마지막으로 space가 여러 번 있는 경우를 전부 단일 space로 정리!
    text = re.sub(r'\s+', ' ', text)

    return text

def preprocess_news(df, column_name = '본문'):
    df[column_name] = df[column_name].apply(cleaning_text)
    return df

In [ ]:
combined_df = preprocess_news(combined_df)
combined_df

In [ ]:
# 누락된 행 없는지 재확인
missing_rows = combined_df[combined_df.iloc[:, 3].isna()]
missing_rows = pd.DataFrame(missing_rows)
missing_rows

In [ ]:
# CSV 파일로 최종 추출!!
combined_df.to_csv("~~데이터셋 취합본~~.csv", encoding="utf-8-sig")